In [35]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
# /share/public/public_models/Qwen3-14B/
# /share/others/public_models/Qwen3-14B/

tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")


Loading checkpoint shards: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


In [36]:
model = model.half()

In [37]:
model = model.to('cuda:0')

In [38]:
model.device

device(type='cuda', index=0)

In [39]:
import torch
dataset = 'alpaca'
model_name = 'Qwen3-8B'
spec_hidden_states_path = f'./train_data/{dataset}_{model_name}_spec_hidden_states.pt'
cur_hidden_states_path = f'./train_data/{dataset}_{model_name}_cur_hidden_states_None_None.pt'
last_hidden_states_path = f'./train_data/{dataset}_{model_name}_true_last_hidden_states_None_None.pt'
spec_hidden_states = torch.load(spec_hidden_states_path)
cur_hidden_states = torch.load(cur_hidden_states_path)
last_hidden_states = torch.load(last_hidden_states_path)
import json
with open(f"./train_data/{dataset}_{model_name}_data_None_None.json", "r") as f:
    data = json.load(f)

In [40]:
cur_hidden_states.shape

torch.Size([40941, 1, 4096])

In [41]:
last_hidden_states.shape

torch.Size([41021, 1, 4096])

In [42]:
len(data['0']['Token'])

919

In [47]:
last_hidden_state = last_hidden_states[920+396]
logits = model.lm_head(last_hidden_state.to(model.device))
token = torch.argmax(logits, dim=-1)
embeddings = model.model.embed_tokens(token)
cur_hidden_state = cur_hidden_states[0]

In [49]:
torch.topk(logits, k=4)

torch.return_types.topk(
values=tensor([[33.8125, 33.8125, 33.5000, 31.0938]], device='cuda:0',
       dtype=torch.float16, grad_fn=<TopkBackward0>),
indices=tensor([[ 2163, 17267,   911,   220]], device='cuda:0'))

In [43]:
data['1']['Token'][395]

{'layer_index': [0,
  4,
  5,
  6,
  10,
  11,
  12,
  14,
  15,
  16,
  19,
  20,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35],
 'similarity': 0.9921875,
 'input_id': 374,
 'output_id': 2163}

In [44]:
data['1']['Token'][396]

{'layer_index': [0, 5, 29, 34, 35],
 'similarity': 0.81494140625,
 'input_id': 17267,
 'output_id': 220}

In [21]:
start_idx_last = 0
start_idx_cur = 0
for key, value in data.items():
    for i in range(len(value['Token'])):
        last_idx = start_idx_last + i
        cur_idx = start_idx_cur + i
        last_hidden_state = last_hidden_states[last_idx]
        logits = model.lm_head(last_hidden_state)
        token = torch.argmax(logits, dim=-1)
        embeddings = model.model.embed_tokens(token)
        cur_hidden_state = cur_hidden_states[cur_idx]
        flag = embeddings == cur_hidden_state
        if not torch.all(flag):
            print(f'{key}_{i} Something wrong')
    start_idx_last += len(value["Token"])+1
    start_idx_cur += len(value['Token'])

1_396 Something wrong
1_429 Something wrong
1_640 Something wrong
3_177 Something wrong
14_304 Something wrong
14_716 Something wrong
15_114 Something wrong
18_246 Something wrong
20_427 Something wrong
21_146 Something wrong
21_306 Something wrong
21_404 Something wrong
26_68 Something wrong
26_678 Something wrong
27_317 Something wrong
29_416 Something wrong
32_389 Something wrong
33_96 Something wrong
33_108 Something wrong
33_121 Something wrong
33_181 Something wrong
33_281 Something wrong
35_475 Something wrong
35_562 Something wrong
42_143 Something wrong
42_159 Something wrong
42_373 Something wrong
42_391 Something wrong
42_722 Something wrong
43_490 Something wrong
45_317 Something wrong
46_246 Something wrong
46_497 Something wrong
47_90 Something wrong
50_514 Something wrong
51_567 Something wrong
54_249 Something wrong
55_154 Something wrong
57_157 Something wrong
58_58 Something wrong
63_256 Something wrong
64_135 Something wrong
66_872 Something wrong
67_41 Something wro

In [18]:
last_hidden_state = last_hidden_states[0]
logits = model.lm_head(last_hidden_state)
token = torch.argmax(logits, dim=-1)
embeddings = model.model.embed_tokens(token)
cur_hidden_state = cur_hidden_states[0]

In [20]:
token

tensor([151667], device='cuda:0')